In [2]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import implicit

"""
Attempt matrix factorization from here:
http://www.benfrederickson.com/matrix-factorization/
"""

foursquare_climate_data = pd.read_csv("./data/foursquare_climate.csv", sep='\t')

/Users/frankyliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (20,21,30,32,34,35,37,49,50,51,52,71,80,97,99) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pd.options.display.max_columns = 999

In [4]:
precip_checkin_count = pd.crosstab(foursquare_climate_data.HOURLYPrecip, foursquare_climate_data.venueId)

In [5]:
temperature_checkin_count = pd.crosstab(foursquare_climate_data.HOURLYDRYBULBTEMPF, foursquare_climate_data.venueId)

In [6]:
import implicit
import scipy

In [7]:
precip_checkin_count

venueId       3fd66200f964a52000e71ee3  3fd66200f964a52000e81ee3  \
HOURLYPrecip                                                       
0.00                                16                         2   
0.01                                 0                         0   
0.02                                 0                         0   
0.03                                 0                         0   
0.04                                 0                         0   
0.05                                 0                         0   
0.06                                 0                         0   
0.07                                 0                         0   
0.08                                 0                         0   
0.09                                 0                         0   
0.10                                 0                         0   
0.11                                 0                         0   
0.12                                 0                         0   
0.13                                 0                         0   
0.14                                 0                         0   
0.15                                 0                         0   
0.16                                 0                         0   
0.17                                 0                         0   
0.18                                 0                         0   
0.19                                 0                         0   
0.20                                 0                         0   
0.21                                 0                         0   
0.22                                 0                         0   
0.23                                 0                         0   
0.24                                 0                         0   
0.25                                 0                         0   
0.26                                 0                         0   
0.27                                 0                         0   
0.28                                 0                         0   
0.29                                 0                         0   
0.33                                 0                         0   
0.37                                 0                         0   
0.38                                 0                         0   
0.42                                 0                         0   
0.45                                 0                         0   
0.53                                 0                         0   
0.54                                 0                         0   
0.55                                 0                         0   
0.70                                 0                         0   
0.75                                 0                         0   
0.85                                 0                         0   
0.93                                 0                         0   
1.05                                 0                         0   
1.76                                 0                         0   

venueId       3fd66200f964a52000f11ee3  3fd66200f964a52001e51ee3  \
HOURLYPrecip                                                       
0.00                                 0                         1   
0.01                                 1                         0   
0.02                                 0                         0   
0.03                                 0                         0   
0.04                                 0                         0   
0.05                                 0                         0   
0.06                                 0                         0   
0.07                                 0                         0   
0.08                                 0                         0   
0.09                                 0                         0   
0.10                                 0                 

In [8]:
pcc = scipy.sparse.csr_matrix(precip_checkin_count.values)

model = implicit.als.AlternatingLeastSquares(factors=50)
model.fit(pcc.astype('double'))


# recommend venue for a precipitation
weather_venues = pcc.T.tocsr()
# 2 is index
recommendation = model.recommend(2, weather_venues)

# find related items
# 2 is index
related = model.similar_items(2)

In [9]:
venues = list(precip_checkin_count)
for rel in related:
    print(venues[rel[0]])

3fd66200f964a52000f11ee3
3fd66200f964a52000e81ee3
3fd66200f964a52001e51ee3
3fd66200f964a5200ce61ee3
3fd66200f964a52001e81ee3
3fd66200f964a52006e91ee3
3fd66200f964a52005e51ee3
3fd66200f964a52008f11ee3
3fd66200f964a52006eb1ee3
3fd66200f964a52006e81ee3


In [10]:
recommendation

[(35, 0.0048275882574618581),
 (22, 0.0036486380700188545),
 (2, 0.0034563890500643318),
 (3, 0.0030690091519966975),
 (34, 0.0027485006951703771),
 (8, 0.0018828037259529425),
 (13, 0.0015705534474815472),
 (39, 0.0015387006558220118),
 (33, 0.001413522342687476),
 (19, 0.0012685437807488376)]

In [11]:
for rec in recommendation:
    print(precip_checkin_count.iloc[rec[0]].name)

0.53
0.22
0.02
0.03
0.45
0.08
0.13
0.75
0.42
0.19


In [12]:
precip_checkin_count.index

Float64Index([ 0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09,  0.1,
              0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19,  0.2, 0.21,
              0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.33, 0.37, 0.38,
              0.42, 0.45, 0.53, 0.54, 0.55,  0.7, 0.75, 0.85, 0.93, 1.05,
              1.76],
             dtype='float64', name='HOURLYPrecip')

In [13]:
tcc = scipy.sparse.csr_matrix(temperature_checkin_count.values)
temperature_model = implicit.als.AlternatingLeastSquares(factors=50)
temperature_model.fit(tcc.astype('double'))


# recommend venue for a temperature
temperature_venues = tcc.T.tocsr()
# 2 is index
venue_recommendations_for_temperature = temperature_model.recommend(2, temperature_venues)

# find related items
# 2 is index
related_venues_temperature = model.similar_items(2)

In [14]:
for rec in venue_recommendations_for_temperature:
    print(temperature_checkin_count.iloc[rec[0]].name)

37.0
29.0
30.0
34.0
33.0
28.0
82.0
35.0
21.0
67.0


In [16]:
temp_venues = list(temperature_checkin_count)
for rel in related_venues_temperature:
    print(temp_venues[rel[0]])

3fd66200f964a52000f11ee3
3fd66200f964a52000e81ee3
3fd66200f964a52001e51ee3
3fd66200f964a5200ce61ee3
3fd66200f964a52001e81ee3
3fd66200f964a52006e91ee3
3fd66200f964a52005e51ee3
3fd66200f964a52008f11ee3
3fd66200f964a52006eb1ee3
3fd66200f964a52006e81ee3


In [17]:
temperature_checkin_count.index

Float64Index([14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0,
              25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0,
              36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0,
              47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0,
              58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0,
              69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0,
              80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0,
              91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 99.0],
             dtype='float64', name='HOURLYDRYBULBTEMPF')